In [1]:
import torch
import spacy
import numpy as np
import torchtext.vocab
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torchtext import data
from transformers import BertTokenizer
import torch
import torch.nn as nn
from sentiment_model.data_utils.tweet_dataset import TweetDataset, pad_batch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Import GloVe Embeddings
glove_twitter = GloVe(name="twitter.27B", dim=50)
tweet_data = load_dataset("SetFit/tweet_sentiment_extraction")
tweet_dataset = TweetDataset(split='train', pretrained_vecs=glove_twitter)
# Instantiate vectors and ensure a 0 vector is inserted for unknown characters
pre_embeds = glove_twitter.vectors
pre_embeds = torch.cat((torch.zeros(2, pre_embeds.shape[1]), pre_embeds))
embedding = nn.Embedding.from_pretrained(pre_embeds, sparse=True, padding_idx=0)


Using custom data configuration SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e
Reusing dataset json (C:\Users\20172613\.cache\huggingface\datasets\json\SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e
Reusing dataset json (C:\Users\20172613\.cache\huggingface\datasets\json\SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
glove_twitter = GloVe(name="twitter.27B", dim=50)

# Instantiate vectors and ensure a 0 vector is inserted for unknown characters
pre_embeds = glove_twitter.vectors
pre_embeds = torch.cat((torch.zeros(1, pre_embeds.shape[1]), pre_embeds))

# Load data:
train_dataset = TweetDataset(split="train", pretrained_vecs=glove_twitter)
valid_dataset = TweetDataset(split="valid", pretrained_vecs=glove_twitter)

# Create data loaders:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=pad_batch)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True, collate_fn=pad_batch)

Using custom data configuration SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e
Reusing dataset json (C:\Users\20172613\.cache\huggingface\datasets\json\SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e
Reusing dataset json (C:\Users\20172613\.cache\huggingface\datasets\json\SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
next(iter(train_loader))[0][1,:]

tensor([   0, 6516,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [8]:
embedding.weight[998]

tensor([ 0.2875,  0.3132, -0.2932,  0.1720, -0.6923, -0.4593,  1.3364,  0.7090,
         0.1212,  0.1148, -0.4850, -0.0886, -3.0154, -0.5402, -1.3260,  0.3948,
         0.1176, -0.1782, -0.3227,  0.2172,  0.0431, -0.4367, -0.5586, -0.4760,
        -0.0952,  0.0032,  0.1192, -0.2364,  1.3234, -0.4509, -0.6584, -0.1387,
         0.2215, -0.3581,  0.2099,  0.0549, -0.0803,  0.4894,  0.1921,  0.4556,
        -1.6420, -0.8332, -0.1297,  0.9651, -0.1821,  0.3773, -0.1962, -0.1223,
        -0.1050,  0.4539])

In [11]:
tweet_dataset.vocab["<pad>"]
#tweet_dataset.vocab.lookup_token(6516)

0

In [18]:
from sentiment_model.model import SentimentNet
from sentiment_model.data_utils.tweet_dataset import pad_batch

train_loader = DataLoader(tweet_dataset, batch_size=32, shuffle=True, collate_fn=pad_batch)
len(tweet_dataset.vocab)

model = SentimentNet()
model_state_dict = torch.load("checkpoints/cool-bee-4/cool-bee-4-epoch-11.pth")['model_state_dict']
model.load_state_dict(model_state_dict)
data, target, text_lengths = next(iter(train_loader))

In [54]:
tweet_dataset.vocab['Hello']

tz = get_tokenizer("spacy", language="en_core_web_sm")
tweet_dataset.vocab['asfj;lsadj;lasdjf']


RuntimeError: Token asfj;lsadj;lasdjf not found and default index is not set

In [46]:
tz("Helllooo how. lkjadf; are yuou\?")

['Helllooo', 'how', '.', 'lkjadf', ';', 'are', 'yuou\\', '?']

In [74]:
from torchtext. vocab import GloVe, vocab

tokens = tz("Helllooo how. lkjadf; are yuou\?".lower())

glove_twitter = GloVe(name = "twitter.27B", dim=50)

In [101]:
glove_twitter.get_vecs_by_tokens()

tensor([-0.4722,  0.6159, -0.6601, -0.1462, -0.4797, -0.5676,  0.4519,  0.4646,
        -0.2835, -0.6584,  0.3741, -0.6437, -3.3311,  0.3133,  0.2019,  0.1715,
         0.6436,  0.2090,  0.0325, -0.1899, -0.8975,  0.9616, -0.2340, -0.4361,
         0.0687, -0.8583,  0.0737, -0.1705,  0.5671,  0.0952, -0.1938, -0.3952,
         0.1049, -0.2117, -1.0626,  0.4975,  0.0692,  0.2275,  0.0324, -0.6410,
        -0.5017,  0.0518,  0.3308,  0.7859,  1.3322, -0.0709,  0.6726,  0.2338,
        -0.7912, -0.6818])

In [90]:
glove_vocab = vocab(glove_twitter.stoi)
unk_token = "<unk>"
unk_index = 0
glove_vocab.insert_token(unk_token, unk_index)
glove_vocab.set_default_index(unk_index)


1193514

In [99]:
a = torch.tensor([1,2,3])
if a is not None:
    print("Yes")

Yes


In [29]:
tz = BertTokenizer.from_pretrained('bert-base-uncased')
print(tz.ids_to_tokens[0])

[PAD]
